# Сервис по составлению меню питания на каждый день

Рабочее название `menu del dia`, `menu_del_dia_bot`

## Описание функционала
**Концепция**
* Это будет телеграм-бот (или email-рассылка), который высылает каждый вечер список блюд и их рецепты на следующий день, а возможно, и примерные расходы не продукты.
* Саначала нужно пройти опрос (в будущем после успешного опробования MVP):
    * на сколько дней в неделю выдать меню (1, 2,..., 7);
    * с какаого дня начинаем готовить (завтра, послезавтра, через неделю)
    * на сколько человек рассчитывать (или сколько порций);
    * едите или не едите мясо, грибы, мясо птицы, морепродукты (креветки, мидии), рыба белая, красная;
    * добавить фрукт на завтрак?
    * какие блюда в обеде: салат, суп, горячее (второе);
    * какие блюда в ужине: салат, суп, горячее (второе).
* В первом сообщении (перед началом 7 дневного периода) будет отправлено на ознакомление:
    * список блюда на каждый день с разделением на завтрак, обед и ужин;
    * список необходимых продуктов и их количество.
    * примерные затраты на питание на неделю (нужен предварительный ввод цен на продукты)
* Дисклеймер про аллергию и т.п.
* Куда писать отзывы и предложения: google-форма для конструктивной критики с заранее заданной структурой (или бот) или прям почту.

**Маинфест**
1. Я решаю свою проблему и мне неважно, что подобный софт уже есть. 
2. Хочу чтобы программа была такой, как я хочу. Хочу чтобы программа 
    * включала в себя только проверенные и опробованные мною блюда;
    * была в удобном для меня виде - телеграмм бот с минималистичным текстом и понятной последовательностью действий;
    * составляла список покупок на неделю вперёд.
    
**Идеи на будущее**
0. Нужно сделать генерировать список рецптов заранее и прописывать их в лог. При перезапуске сервера нужно начать с того же места и не отправлять заново то, что уже отправлено.
1. Возможность ставить лайки и дислайки. Куда-то сохранять эти реакции и ещё какую-нибудь добровольно полученную информацию о пользователях (пол, возраст, дети, регион)Потом давать чаще блюда с лайками.
2. Расчёт трат на продукты.
3. ML: рекомендации блюд. Это как в IVI: выбери 5...10 фильмов, и сделаем тебе рекомендации. Или сразу рекомендовать блюда, которые понравились таким же пользователям или ещё как-нибудь.
4. Время подготовки продуктов, время приготовления блюда. Можно поэкспериментировать и сделать модель, напрмиер `время = количество лукавиц * время резки одной лукавицы + количество каортофелин * время резки одной картофелины`  
5. Подсчёт калорий (это в самый хваост списка, т.к. мне это не нужно).
6. Возможность повторов блюд на неделе (True / False) или конкретное заданеи возможного количества повторов в виде числа.
7. Почитать https://journal.tinkoff.ru/list/chto-est/

## Типы блюд для MVP

Классификация по времени:
* Завтраки
* Обеды и ужины

Классификация по типу
* Салаты
* Супы
* Горячее

## Алгоритм для MVP

1. В конце каждой недели генерируется меню на следующую неделю.

   - Из базы выбирается n_days=7 случайных блюд
       - из категории "Завтраки" (`meal_type.meal_type_name = 'Завтрак'`)
       - из категории "Обеды" (`meal_type.meal_type_name = 'Обед или ужин'`)
       - из категории "Ужины" (`meal_type.meal_type_name = 'Обед или ужин'`)
   - Формируем списки из id блюд и записываем их во временный файл (он нужен, чтобы восстановить план на неделю в случае отключения сервера и его перезагрузки). В файле проставляем дату его изменения (?).
   - Если дата изменения файла не старше 7 дней, то используем его в качестве источника для формирования запросов, иначе генерируем новый.
3. Высылаем сообщения.
    1. Высылается меню в виде списка блюд: `ЗАВТРАКИ: ... ОБЕДЫ: УЖИНЫ: ...` 
    2. Высылается список покупок в виде списка продуктов и количества.
    3. Высылается полное меню на завтра: тип, название, рецепт.
    4. Ждём 24 часа и высылаем меню на следующий день и так до следующего конца недели.

- #TODO: Нужен UI: с какого дня недели начинать
- #TODO: Нужен UI: Нужно выбрать, на какие именно дни присылать рецепты, необязательно гототвить 7 дней в неделю (`n_days`).
- #TODO: Нужен UI: Нужно выбрирать, какие прдукты пользователь ест или не ест (`preferred_ingredients`).
- #TODO: Разные блюда на обед и ужин.
- #TODO: Вместо временного файла с планом на неделю записывать этот план в базу данных пользователей. 
- #TODO: 
- #TODO: 
- #TODO:

## Добыча рецептов

Всё вручную, т.к. нужно опробовать, переделать и переписать.

## База данных рецептов

### Создание

In [15]:
import pandas as pd
import sqlite3

In [16]:
path = '..\\data\\recipies.xlsx'

In [17]:
df_recipes = pd.read_excel(path, sheet_name='recipe')
df_recipes.sample(3)

,recipe_id,food_category_id,meal_type_id,recipe_name,prep_time,cook_time,recipe_steps,link_to_original
39,40,3,2,Запеченые стейки из форели с пюре,5.0,30.0,Нарезать лук полукольцами и выложить в форму д...,https://lafoy.ru/steyki-foreli-v-duhovke-recep...
18,19,2,2,Салат «Оливье»,NaN,30.0,Отварить мясо. Отварить картофель в кожуре. От...,https://www.say7.info/cook/recipe/194-Salat-Ol...
5,6,3,1,Гречневая каша с молоком,1.0,30.0,Гречневую крупу при необходимости перебрать — ...,https://www.russianfood.com/recipes/recipe.php...


In [18]:
df_food_category = pd.read_excel(path, sheet_name='food_category')
df_food_category.sample(3)

,food_category_id,food_category_name
3,4,Десерт
0,1,Суп
2,3,Основное блюдо


In [19]:
df_meal_type = pd.read_excel(path, sheet_name='meal_type')
df_meal_type.sample(3)

,meal_type_id,meal_type_name
3,4,Обед
2,3,Любое
4,5,Ужин


In [20]:
df_ingredients = pd.read_excel(path, sheet_name='ingredients')
df_ingredients.sample(3)

,ingredient_id,ingredient_name
17,18.0,Мука пшеничная
42,43.0,Орегано сушеный
85,86.0,Зира


In [21]:
df_recipe_ingredients = pd.read_excel(
    path, sheet_name='recipe_ingredients',
    usecols=['recipe_id', 'ingredient_id', 'ingredient_qty', 'measurement_id']
)
df_recipe_ingredients.sample(3)

,recipe_id,ingredient_id,ingredient_qty,measurement_id
161,26,1,35.0,1
192,28,44,0.0,9
209,30,9,0.0,9


In [24]:
# Create SQLite database connection
conn = sqlite3.connect('..\\data\\recipes.db')

# Write DataFrames to SQLite tables
df_recipes.to_sql('recipe', conn, index=False, if_exists='replace')
df_food_category.to_sql('food_category', conn, index=False, if_exists='replace')
df_meal_type.to_sql('meal_type', conn, index=False, if_exists='replace')
df_ingredients.to_sql('ingredients', conn, index=False, if_exists='replace')
df_recipe_ingredients.to_sql('recipe_ingredients', conn, index=False, if_exists='replace')
# df_measurements.to_sql('measurements', conn, index=False, if_exists='replace')

# Close database connection
conn.close()

In [26]:
# Example query: Retrieve all recipes with their names and preparation times

# Create SQLite database connection
conn = sqlite3.connect('..\\data\\recipes.db')

# Create a cursor object to interact with the database
cursor = conn.cursor()
cursor.execute('SELECT recipe_id, recipe_name FROM recipe')

# Fetch all rows from the result set
rows = cursor.fetchall()

# Print the fetched rows
for i, row in enumerate(rows):
    print(i, row)

# Close cursor and database connection
cursor.close()
conn.close()

0 (1, 'Хлопья кукурузные с молоком')
1 (2, 'Гранола с йогуртом')
2 (3, 'Оладьи без яиц')
3 (4, 'Манная каша на молоке')
4 (5, 'Рисовая каша на молоке')
5 (6, 'Гречневая каша с молоком')
6 (7, 'Овсяная каша на молоке')
7 (8, 'Яйца в смятку')
8 (9, 'Омлет')
9 (10, 'Сырники жареные')
10 (11, 'Сырники печёные')
11 (12, 'Творожная запеканка')
12 (13, 'Пшённая каша на молоке')
13 (14, 'Блины со сметаной')
14 (15, 'Салат «Витаминный»')
15 (16, 'Салат «Греческий»')
16 (17, 'Салат «Винегрет»')
17 (18, 'Салат «Цезарь»')
18 (19, 'Салат «Оливье»')
19 (20, 'Салат «Мимоза»')
20 (21, 'Салат «Сельдь под шубой»')
21 (22, 'Салат свекольный с чесноком')
22 (23, 'Суп-пюре с шампиньонами и картофелем')
23 (24, 'Томатный суп с фрикадельками')
24 (25, 'Борщ с индейкой')
25 (26, 'Суп с фрикадельками из индейки и шпинатом')
26 (27, 'Суп с фрикадельками из индейки и фасолью')
27 (28, 'Суп с курицей, стручковой фасолью и грибами')
28 (29, 'Уха по-фински')
29 (30, 'Тыквенный крем-суп')
30 (31, 'Суп-пюре с броккол

### Работа с базой

In [1]:
import os.path, time
import sqlite3
import json

In [3]:
# Selecting dishes for a week
connection = sqlite3.connect('../db/recipes.db')
cursor = connection.cursor()

meal_types = ['Завтрак', 'Обед или ужин', 'Обед или ужин']
for meal_type in meal_types:
    r_id = 5
    query = f'SELECT * FROM recipe WHERE recipe_id = {r_id}'

cursor.execute(query)
# Fetch all rows from the result set
rows = cursor.fetchall()
print(rows)

# Close cursor and database connection
cursor.close()
connection.close()

[(5, 3, 1, 'Рисовая каша на молоке', 3.0, 30.0, 'Промыть рис в холодной воде. Высыпать промытый рис в кастрюлю. Добавить воду. Довести воду до кипения. Добавить соль. Перемешать. Варить до выкипания воды (примерно 10 минут). Добавить молоко. Уменьшить огонь / мощность плиты. Варить 10 минут, периодически помешивая. Добавить сахар и сливочное масло. Выключить плиту. Накрыть кастрюлю крышкой. Дать настояться 10 минут.', 'https://www.russianfood.com/recipes/recipe.php?rid=160677')]


In [4]:
print("last modified: %s" % time.ctime(os.path.getmtime(file)))
print("created: %s" % time.ctime(os.path.getctime(file)))

NameError: name 'file' is not defined

In [13]:
# сколько порций? 
n = 4
recipe_name = 'Томатный суп с фрикадельками'
for ing, mass, in zip(db[recipe_name]['ingridients'], db[recipe_name]['ingredient_quantity']):
    print(ing, '-', n * mass, 'г')

фарш из мяса индейки - 125.0 г
картофель - 125.0 г
подмидор - 87.5 г
болгарский перец - 50.0 г
томатная паста - 37.5 г
лук репчатый - 37.5 г
молотая паприка - 1.75 г
сушёный укроп, лук и петрушка - 1.75 г
соль - 0 г
перец горошком - 0 г
растительное масло - 0 г


## Первая версия (MVP)

**Алгоритм для MVP**

1. В конце каждой недели генерируется меню на следующую неделю.

   - Из базы выбирается n_days=7 случайных блюд
       - из категории "Завтраки" (`meal_type.meal_type_name = 'Завтрак'`)
       - из категории "Обеды" (`meal_type.meal_type_name = 'Обед или ужин'`)
       - из категории "Ужины" (`meal_type.meal_type_name = 'Обед или ужин'`)
   - Формируем списки из id блюд и записываем их во временный файл (он нужен, чтобы восстановить план на неделю в случае отключения сервера и его перезагрузки). В файле проставляем дату его изменения (?).
   - Если дата изменения файла не старше 7 дней, то используем его в качестве источника для формирования запросов, иначе генерируем новый.
3. Высылаем сообщения.
    1. Высылается меню в виде списка блюд: `ЗАВТРАКИ: ... ОБЕДЫ: УЖИНЫ: ...` 
    2. Высылается список покупок в виде списка продуктов и количества.
    3. Высылается полное меню на завтра: тип, название, рецепт.
    4. Ждём 24 часа и высылаем меню на следующий день и так до следующего конца недели.

- #TODO: Нужен UI: с какого дня недели начинать
- #TODO: Нужен UI: Нужно выбрать, на какие именно дни присылать рецепты, необязательно гототвить 7 дней в неделю (`n_days`).
- #TODO: Нужен UI: Нужно выбрирать, какие прдукты пользователь ест или не ест (`preferred_ingredients`).
- #TODO: Разные блюда на обед и ужин.
- #TODO: Вместо временного файла с планом на неделю записывать этот план в базу данных пользователей. 
- #TODO: 
- #TODO: 
- #TODO:

In [5]:
import sqlite3
import random
from datetime import datetime, timedelta
import json
import os

db_path = '../data/recipes.db'

### Выполнение 1 раз перед новой неделей

https://stackoverflow.com/questions/43668806/python-how-to-automate-script-to-run-every-day-at-specific-time#43669809
Выполнение этих скриптов должно быть раз в недею. Сделать cronjob.

Here's how you can do it:
    1. Open your terminal.
    2. Type `crontab -e` and press Enter to edit your cron jobs.
    3. Add the following line at the end of the file: `0 10 * * 0 /path/to/python3 /path/to/menu_generator.py >> /path/to/menu_generation.log 2>&1`. Replace `/path/to/python3` with the path to your Python 3 interpreter, and `/path/to/menu_generator.py` with the path to your Python script menu_generator.py.
    4. Save and exit the file.


**Syntax Explanation**

The cronjob syntax consists of five fields, representing different time units and scheduling intervals. Here's what each field in the example means:

1. `Minute (0)`: Specifies the minute when the command will be executed. In this case, it's set to `0`, which means the command will be executed at the start of the hour.
2. `Hour (10)`: Specifies the hour when the command will be executed. In this case, it's set to `10`, which means the command will be executed at 10:00 AM.
3. `Day of the Month (*)`: Specifies the day of the month when the command will be executed. Here, it's set to `*`, which means any day of the month is acceptable. 
4. `Month ( *)`: Specifies the month when the command will be executed. Here, it's set to `*`, which means any month is acceptable.
5. `Day of the Week (0)`: Specifies the day of the week when the command will be executed. In this case, it's set to `0`, which represents Sunday.


Putting it all together, the cronjob syntax `0 10 * * 0` means the command will be executed at 10:00 AM every Sunday. 
So, in our example, the cronjob is set to run the command `/path/to/python3 /path/to/menu_generator.py` every Sunday at 10:00 AM.     
`>>`: Redirects the standard output (stdout) stream to a file. This appends the output to the end of the file, preserving existing content. `/path/to/menu_generation.log`: Specifies the path to the log file where the output will be written. `2>&1`: Redirects the standard error (stderr) stream to the same location as the standard output. This ensures that both stdout and stderr are captured in the log file.



#### Формирование списка рецептов и запись в файл

In [6]:
# Define SQLite query
GET_RANDOM_RECIPES = """
SELECT r.recipe_id
FROM recipe r
JOIN meal_type m ON r.meal_type_id = m.meal_type_id
WHERE {}
ORDER BY RANDOM() LIMIT ?
"""

def lst_to_str(lst: list) -> str:
    """
    Convert a list of elements to a string representation.

    Args:
        lst (list): The list of elements to be converted to a string.

    Returns:
        str: A string representation of the input list with elements enclosed in parentheses
             and separated by commas.

    Examples:
        >>> lst_to_str([40, 41, 31, 43, 20, 23, 38])
        '(40, 41, 31, 43, 20, 23, 38)'
        >>> lst_to_str(['apple', 'banana', 'cherry'])
        "('apple', 'banana', 'cherry')"
        >>> lst_to_str([])
        '()'

    Note:
        - If the input list is empty, an empty string with parentheses '()' will be returned.
        - Elements in the list will be converted to their string representations using the str() function.
    """
    return '(' + ', '.join(map(str, lst)) + ')'

# Function to generate menu for the week
def generate_weekly_menu():
    """
    Generate a weekly menu by selecting random recipes for each meal type.

    This function queries the database to retrieve random recipe IDs for each
    meal type (breakfast, lunch, and dinner) based on specified conditions.
    The conditions ensure that for breakfast, only breakfast recipes are
    selected, while for lunch and dinner, recipes can be chosen from a broader
    category. Additionally, dinner recipes that have already been selected for
    lunch in the week are excluded.

    Returns:
        dict: A dictionary containing meal types as keys and lists of recipe
              IDs as values. Each list represents the selected recipes for
              the corresponding meal type. The dictionary also includes a
              'week_number' key, indicating the number of the current week
              in the year, and a 'year' key, indicating the current year.


    Example usage:
        menu = generate_weekly_menu()
        print(menu)
    """
    menu = dict()
    recipe_ids = list()
    # Meal types and corresponding conditions
    meal_types = ['breakfast', 'lunch', 'dinner']
    where_conditions = [
        "m.meal_type_name = 'Завтрак'",
        "m.meal_type_name = 'Обед или ужин'",
        f"m.meal_type_name = 'Обед или ужин' AND r.recipe_id NOT IN {lst_to_str(recipe_ids)}" 
    ]

    for meal_type, condition in zip(meal_types, where_conditions):
        # Execute SQL query to fetch random recipe IDs
        query = GET_RANDOM_RECIPES.format(condition)
        # Fetch recipe_ids and extract IDs from tuples
        recipe_ids = [id[0] for id in cursor.execute(query, (7,)).fetchall()]
        # Store the selected recipe IDs for the current meal type
        menu[meal_type] = recipe_ids

    # Get the current week number and year
    current_date = datetime.now()
    week_number = current_date.isocalendar()[1]
    year = current_date.year

    # Add week number and year to the menu dictionary
    menu['week_number'] = week_number
    menu['year'] = year
    # TODO: Add user Id to the output dictionary
    
    return menu


# Initialize SQLite database connection
with sqlite3.connect(db_path) as connection:
    cursor = connection.cursor()
    menu = generate_weekly_menu()

menu

{'breakfast': [12, 47, 7, 1, 13, 10, 14],
 'lunch': [40, 17, 23, 22, 44, 28, 34],
 'dinner': [42, 30, 19, 20, 16, 41, 17],
 'week_number': 18,
 'year': 2024}

In [7]:
# Function to write menu to JSON file
def write_menu_to_json(menu, path='weekly_menu.json'):
    """
    Write the menu dictionary to a JSON file.

    Args:
        menu (dict): A dictionary containing the menu data.
        path (str): Optional. The path to the JSON file. Default is 'weekly_menu.json'.

    Example usage:
        write_menu_to_json(menu, 'menu.json')
    """
    with open(path, 'w') as file:
        json.dump(menu, file, indent=4)

write_menu_to_json(menu)

#### Формирование списка блюд и списка продуктов. Отправка сообщения в Telegram

In [66]:
import sqlite3
from datetime import datetime
import json

# Number of persons
N_PERSONS = 4  #TODO: Make it a variable and be able to be changed in UI
# Number of days
N_DAYS = 7 #TODO: Make it a variable and be able to be changed in UI
# Path to the database
DB_PATH = '..\\data\\recipes.db'
# Path to json-file containing the recipe information
JSON_MENU_PATH = '..\\data\\weekly_menu.json'

meal_types = ['breakfast', 'lunch', 'dinner']
meal_types_ru = ['Завтрак', 'Обед', 'Ужин']

def lst_to_str(lst: list) -> str:
    """
    Convert a list of elements to a string representation.

    Args:
        lst (list): The list of elements to be converted to a string.

    Returns:
        str: A string representation of the input list with elements enclosed 
             in parentheses and separated by commas.

    Examples:
        >>> lst_to_str([40, 41, 31, 43, 20, 23, 38])
        '(40, 41, 31, 43, 20, 23, 38)'
        >>> lst_to_str(['apple', 'banana', 'cherry'])
        "('apple', 'banana', 'cherry')"
        >>> lst_to_str([])
        '()'

    Note:
        - If the input list is empty, an empty string with parentheses '()' 
          will be returned.
        - Elements in the list will be converted to their string 
          representations using the str() function.
    """
    return '(' + ', '.join(map(str, lst)) + ')'

# Define SQLite query
GET_RANDOM_RECIPES = """
SELECT r.recipe_id
FROM recipe r
JOIN meal_type m ON r.meal_type_id = m.meal_type_id
WHERE {}
ORDER BY RANDOM() LIMIT ?
"""

def generate_weekly_menu():
    """
    Generate a weekly menu by selecting random recipes for each meal type.

    This function queries the database to retrieve random recipe IDs for each
    meal type (breakfast, lunch, and dinner) based on specified conditions.
    The conditions ensure that for breakfast, only breakfast recipes are
    selected, while for lunch and dinner, recipes can be chosen from a broader
    category. Additionally, dinner recipes that have already been selected for
    lunch in the week are excluded.

    Returns:
        dict: A dictionary containing meal types as keys and lists of recipe
              IDs as values. Each list represents the selected recipes for
              the corresponding meal type. The dictionary also includes a
              'week_number' key, indicating the number of the current week
              in the year, and a 'year' key, indicating the current year.


    Example usage:
        menu = generate_weekly_menu()
        print(menu)
    """
    menu = dict()
    recipe_ids = list()
    # Meal types and corresponding conditions
    where_conditions = [
        "m.meal_type_name = 'Завтрак'",
        "m.meal_type_name = 'Обед или ужин'",
        "m.meal_type_name = 'Обед или ужин'" +\
              f"AND r.recipe_id NOT IN {lst_to_str(recipe_ids)}"
    ]

    for meal_type, condition in zip(meal_types, where_conditions):
        # Execute SQL query to fetch random recipe IDs
        query = GET_RANDOM_RECIPES.format(condition)
        # Fetch recipe_ids and extract IDs from tuples
        recipe_ids = [id[0] for id in cursor.execute(query, (N_DAYS,)).fetchall()]
        # Store the selected recipe IDs for the current meal type
        menu[meal_type] = recipe_ids

    # Get the current week number and year
    current_date = datetime.now()
    week_number = current_date.isocalendar()[1]
    year = current_date.year

    # Add week number and year to the menu dictionary
    menu['week_number'] = week_number
    menu['year'] = year
    # TODO: Add user Id to the output dictionary

    return menu


# Function to write menu to JSON file
def write_menu_to_json(menu, path='weekly_menu.json'):
    """
    Write the menu dictionary to a JSON file.

    Args:
        menu (dict): A dictionary containing the menu data.
        path (str): Optional. The path to the JSON file. Default is 'weekly_menu.json'.

    Example usage:
        write_menu_to_json(menu, 'menu.json')
    """
    with open(path, 'w') as file:
        json.dump(menu, file, indent=4)

MAKE_LIST_OF_PRODUCTS = """
SELECT 
    i.ingredient_name, 
    SUM(ri.ingredient_qty) * ? AS total_qty, 
    m.measurement_name 
FROM 
    recipe_ingredients ri
JOIN 
    ingredients i ON ri.ingredient_id = i.ingredient_id
JOIN 
    measurements m ON ri.measurement_id = m.measurement_id
WHERE {}
GROUP BY 
    ri.ingredient_id, m.measurement_name
ORDER BY 
    i.ingredient_name
"""

def make_list_of_products(recipe_ids: list) -> str:
    """
    Uses MAKE_LIST_OF_PRODUCTS query and returns multiple line string 
    of ingredients with their quantities. Makes a list of products to buy.

    Args:
        recipe_ids (list): list of recipe IDs

    Returns:
        str: multiple line string with ingredients and their quantities and 
             measurements
    """
    condition = "ri.recipe_id IN {}".format(lst_to_str(recipe_ids))
    query = MAKE_LIST_OF_PRODUCTS.format(condition)

    # Execute the SQL query to retrieve data
    data = cursor.execute(query, (N_PERSONS,)).fetchall()

    # Constructing the string representation of ingredients with their quantities and measurements
    result_str = ""
    for row in data:
        ingredient_name, total_qty, measurement_name = row
        result_str += f"{ingredient_name}: {total_qty} {measurement_name}\n"

    return result_str

MAKE_LIST_OF_DISHES = """
SELECT recipe_name
FROM recipe
WHERE {}
ORDER BY recipe_name
"""

def make_list_of_dishes(recipe_ids: list) -> str:
    """
     Uses MAKE_LIST_OF_DISHES query and returns multiple line string 
    of ingredients with their quantities.  Makes a list of dishes for a week

   Args:
        recipe_ids (list): list of recipe IDs

    Returns:
        str: multiple line string with dishes
    """
    condition = "recipe_id IN {}".format(lst_to_str(recipe_ids))
    query = MAKE_LIST_OF_DISHES.format(condition)

    # Execute the SQL query to retrieve data
    data = cursor.execute(query).fetchall()

    # Constructing the string representation of ingredients with their quantities and measurements
    result_str = ""
    for row in data:
        recipe_name = row[0]
        result_str += f"{recipe_name}\n"

    return result_str


# Initialize SQLite database connection
# TODO: развести процессы формирования menu и отправки сообщений в telegram. Сделать weekly_message_sender.py
with sqlite3.connect(DB_PATH) as connection:
    cursor = connection.cursor()
    print('Generating menu...')  #TODO: писать в логи
    menu = generate_weekly_menu()
    print('Menu generation complete') #TODO: писать в логи

    print('Generating message with dishes list...') #TODO: писать в логи
    message_with_recipe_names = ''
    for meal_type, meal_type_ru in zip(meal_types, meal_types_ru):
        message_with_recipe_names += meal_type_ru.upper() + '\n'
        message_with_recipe_names += make_list_of_dishes(menu[meal_type]) + '\n'
    print('Message generation complete') #TODO: писать в логи

    print('Generating message with buying list...') #TODO: писать в логи
    recipe_ids = []
    for meal_type in meal_types:
        recipe_ids += menu[meal_type]
    message_with_buying_list = make_list_of_products(recipe_ids) + '\n'
    print('Message generation complete') #TODO: писать в логи

print('Sending messages...') #TODO: писать в логи
print('\nСписок блюд')
print(message_with_recipe_names) # TODO: send to telegram
print('Список покупок')
print(message_with_buying_list) # TODO: send to telegram
print('Writing recipe IDs to temporary json-file')
write_menu_to_json(menu, path=JSON_MENU_PATH)
print()


Generating menu...
Menu generation complete
Generating message with dishes list...
Message generation complete
Generating message with buying list...
Message generation complete
Sending messages...

Список блюд
ЗАВТРАК
Блины со сметаной
Гречневая каша с молоком
Рисовая каша на молоке
Сырники печёные
Творожная запеканка
Хлопья кукурузные с молоком
Яйца в смятку

ОБЕД
Капуста с куриным филе
Паста карбонара
Салат «Мимоза»
Салат свекольный с чесноком
Суп с курицей, стручковой фасолью и грибами
Томатный суп с фрикадельками
Тыквенный крем-суп

УЖИН
Борщ с индейкой
Ризотто с морепродуктами
Салат «Цезарь»
Солянка
Суп с фрикадельками из индейки и фасолью
Томатный суп с фрикадельками
Уха по-фински


Список покупок
Бекон: 172.0 г
Болгарский перец: 50.0 г
Варёный яичный желток: 1.0 шт.
Вино белое сухое: 200.0 мл
Вода: 6260.0 мл
Горчица: 4.0 столовая ложка без горки
Гречневая крупа: 420.0 г
Зелень (лук, петрушка, укроп): 0.0 по вкусу / по необходимости
Капуста белокочанная: 640.0 г
Картофель: 1025.

### Запуск каждый день вечером. Отправка рецептов на следующий день

In [ ]:
# Function to send messages
def send_messages():
    # Read menu from JSON file
    with open('weekly_menu.json', 'r') as file:
        menu = json.load(file)

    # Send full menu for tomorrow
    tomorrow_menu = get_tomorrow_menu()
    send_tomorrow_menu(tomorrow_menu)

# Function to extract ingredients from menu
def extract_ingredients(menu):
    # Extraction of ingredients from recipes in the menu
    pass

# Function to get full menu for tomorrow
def get_tomorrow_menu():
    # Fetching of menu for tomorrow from the weekly menu
    with open('weekly_menu.json', 'r') as file:
        weekly_menu = json.load(file)
    pass

# Function to send full menu for tomorrow
def send_tomorrow_menu(tomorrow_menu):
    print("Sending menu for tomorrow:")
    print(tomorrow_menu)



# Run the main function
main()

# Close database connection
conn.close()


In [81]:
day_number

4

In [4]:
import sqlite3
import random
from datetime import datetime, timedelta
import json
import os

db_path = '../db/recipes.db'

In [ ]:
# Initialize SQLite database connection
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Define SQLite queries
GET_RANDOM_RECIPES = '''
SELECT r.recipe_name, r.recipe_steps, r.link_to_original, m.meal_type_name
FROM recipe r
JOIN meal_type m ON r.meal_type_id = m.meal_type_id
WHERE m.meal_type_name = ?
ORDER BY RANDOM() LIMIT ?
'''

# Function to generate menu for the week
def generate_weekly_menu():
    menu = {}
    for meal_type in ['Завтрак', 'Обед или ужин', 'Обед или ужин']:
        recipes = cursor.execute(GET_RANDOM_RECIPES, (meal_type, 7)).fetchall()
        menu[meal_type] = recipes
    return menu

# Function to write menu to JSON file
def write_menu_to_json(menu):
    with open('weekly_menu.json', 'w') as file:
        json.dump(menu, file, indent=4)

# Function to send messages
def send_messages():
    # Read menu from JSON file
    with open('weekly_menu.json', 'r') as file:
        menu = json.load(file)

    # Send menu as a list of dishes
    send_menu(menu)

    # Send shopping list
    send_shopping_list(menu)

    # Send full menu for tomorrow
    tomorrow_menu = get_tomorrow_menu()
    send_tomorrow_menu(tomorrow_menu)

# Function to send menu as a list of dishes
def send_menu(menu):
    print("Sending weekly menu:")
    print(menu)

# Function to send shopping list
def send_shopping_list(menu):
    print("Sending shopping list:")
    # Extract ingredients from menu and send them
    ingredients = extract_ingredients(menu)
    print(ingredients)

# Function to extract ingredients from menu
def extract_ingredients(menu):
    # Simulated extraction of ingredients from recipes in the menu
    return ['Ingredient 1', 'Ingredient 2', 'Ingredient 3']

# Function to get full menu for tomorrow
def get_tomorrow_menu():
    # Simulated fetching of menu for tomorrow from the generated weekly menu
    with open('weekly_menu.json', 'r') as file:
        weekly_menu = json.load(file)
    return weekly_menu.get('Обед или ужин', '')

# Function to send full menu for tomorrow
def send_tomorrow_menu(tomorrow_menu):
    print("Sending menu for tomorrow:")
    print(tomorrow_menu)

# Check if weekly menu file exists and is up to date
def is_weekly_menu_up_to_date():
    if not os.path.exists('weekly_menu.json'):
        return False
    else:
        last_modified_time = os.path.getmtime('weekly_menu.json')
        last_modified_date = datetime.fromtimestamp(last_modified_time).date()
        current_date = datetime.now().date()
        return (current_date - last_modified_date) <= timedelta(days=7)

# Main function
def main():
    if not is_weekly_menu_up_to_date():
        weekly_menu = generate_weekly_menu()
        write_menu_to_json(weekly_menu)
    send_messages()

# Run the main function
main()

# Close database connection
conn.close()


## Запуск скрипта по распианию

https://www.geeksforgeeks.org/scheduling-python-scripts-on-linux/

To automate the script using CRON, you can modify the script to execute the desired functionality and then create a CRON job to run the script at specific intervals. Here's the modified version of the Python script that should be run using CRON:


### Setting Up CRON Job:

1. Open your terminal or SSH into your server.
2. Open the CRON table for editing using the command:
   ```
   crontab -e
   ```
3. Add a new CRON job to schedule the script to run every day at a specific time. For example, to run the script every day at 6 PM, add the following line to the CRON table:
   ```
   0 18 * * * /path/to/python3 /path/to/recipe_bot.py
   ```
   Replace `/path/to/python3` with the path to your Python 3 interpreter and `/path/to/recipe_bot.py` with the full path to your Python script.
4. Save and exit the editor.

Now, the CRON job will run the `recipe_bot.py` script every day at 6 PM. Make sure to adjust the CRON schedule and file paths according to your requirements and server setup.

In this script, sent recipes are logged in a file named recipe_bot.log along with timestamps. The get_last_sent_timestamp function reads the log file to retrieve the timestamp of the last sent recipe, allowing the script to resume sending recipes from the correct point.

Make sure to adjust the log file path, log format, and content according to your specific requirements. This approach simplifies the setup and provides a straightforward way to maintain a history of sent recipes.

In [ ]:
import os
import random
import logging
from datetime import datetime, timedelta
from telegram import Bot

# Configure logging
logging.basicConfig(filename='recipe_bot.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Define SQLite queries
GET_WEEKLY_RECIPES = 'SELECT recipe_name FROM recipes ORDER BY RANDOM() LIMIT 7'
GET_DAILY_RECIPES = 'SELECT recipe_name FROM recipes ORDER BY RANDOM() LIMIT 3'

# Define Telegram bot token and chat ID
TELEGRAM_TOKEN = 'YOUR_TELEGRAM_BOT_TOKEN'
CHAT_ID = 'YOUR_CHAT_ID'

# Helper function to send messages
def send_message(recipes):
    bot = Bot(token=TELEGRAM_TOKEN)
    message = "\n".join(recipes)
    bot.send_message(chat_id=CHAT_ID, text=message)
    # Log sent recipes
    logging.info(f'Sent recipes: {recipes}')

# Get last sent recipe timestamp from the log file
def get_last_sent_timestamp():
    try:
        with open('recipe_bot.log', 'r') as log_file:
            lines = log_file.readlines()
            if lines:
                last_line = lines[-1]
                timestamp_str = last_line.split(' - ')[0]
                return datetime.strptime(timestamp_str, '%Y-%m-%d %H:%M:%S,%f')
    except FileNotFoundError:
        pass
    return None

# Get weekly recipes and send them
def send_weekly_recipes():
    recipes = cursor.execute(GET_WEEKLY_RECIPES).fetchall()
    recipes = [recipe[0] for recipe in recipes]
    send_message(recipes)

# Get daily recipes and send them
def send_daily_recipes():
    recipes = cursor.execute(GET_DAILY_RECIPES).fetchall()
    recipes = [recipe[0] for recipe in recipes]
    send_message(recipes)

# Check current day and send appropriate recipes
current_day = datetime.now().weekday()

last_sent_timestamp = get_last_sent_timestamp()

if current_day == 6:  # Sunday (Index 6 in Python's datetime)
    send_weekly_recipes()
else:
    if last_sent_timestamp is None or (datetime.now() - last_sent_timestamp).days > 1:
        send_daily_recipes()


## Восстановаление после аварийного завершения программы

Нужно подготовиться к выключению сервера.

`random_state` должно быть задано конкретным числом

Можно каждое 
* расписание на неделю
* каждое отправленное сообщение 
логировать и ввести счётчики отработавших циклов. 

После перезапуска програмы отпарвить сообщение и сбое на сервере и начать пролграмму с тех значений счётчика, на которых программа остановилась в прошлый раз - эти значения можно взять из логов.

А как быть когда много пользователей? В логах прописывать ещё и id пользователей

## Ссылки, которые помогли
* https://realpython.com/beautiful-soup-web-scraper-python/
* https://dbeaver.com/2022/06/23/import-data-with-dbeaver/
* https://stackoverflow.com/questions/38463361/creating-a-recipe-database
* https://stackoverflow.com/questions/54005899/how-do-i-properly-design-a-data-model-for-recipe-ingredient-using-mongodb
* https://infoeda.com/skolko-v-stolovoj-lozhke-gramm-i-millilitrov-tablica.html

## Чему научился:
1. проектирование базы данных;
2. использвоание SQL-запросов к базе данных в Python-коде;
3. использование DBeaver в качестве СУБД;
